TimeEcho: Immersive Historical Storytelling with GenAI

Welcome to **TimeEcho**, a generative AI-powered project that transforms historical events into immersive, first-person experiences.

**Goal:**  
To bring history to life using GenAI by combining retrieval-augmented generation (RAG), embeddings, image/audio generation, and persona-based narratives.

**Key Capabilities Used:**
- 🔍 Retrieval-Augmented Generation (RAG)
- 🧠 Gemini Embeddings
- 🖼️ Image & Visual Understanding
- 🎙️ Audio Narration with Emotion Cues
- 🧾 Structured JSON Outputs
- 🔁 Few-Shot Prompting
- 💡 Dynamic Agents for Query Handling

Install Required Libraries & Import Dependencies

We begin by installing essential packages for working with:
- Gemini GenAI
- Wikipedia data scraping
- FAISS for vector search
- Audio/image generation

In [1]:
!pip install google-generativeai 
!pip install wikipedia-api
!pip install faiss-cpu

  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia-api: filename=Wikipedia_API-0.8.1-py3-none-any.whl size=15384 sha256=bc8c869c3d0b3c81115909d7486e6fb54f64745e705023b89fceeb101f849d23
  Stored in directory: /root/.cache/pip/wheels/0b/0f/39/e8214ec038ccd5aeb8c82b957289f2f3ab2251febeae5c2860
Successfully built wikipedia-api
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 49.4 MB/s eta 0:00:00


In [2]:
import os
import json
import numpy as np
import pandas as pd
import wikipediaapi
from typing import List, Dict, Any
import google.generativeai as genai
from IPython.display import Markdown, display
import faiss
import pickle
from kaggle_secrets import UserSecretsClient

In [3]:

GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_AI_Studio_key")

genai.configure(api_key=GOOGLE_API_KEY)

model = genai.GenerativeModel(model_name="gemini-2.0-flash")
response = model.generate_content("Hello, are you ready to help create TimeEcho?")
print(response.text)

Yes, I am ready to help create TimeEcho! 

To get started, I need a better understanding of what TimeEcho is.  Please tell me more about:

*   **What is the purpose of TimeEcho?** What problem does it solve, or what need does it fulfill?
*   **What are the core features of TimeEcho?**  What can users *do* with it?
*   **Who is the target audience?** Who are you trying to reach with this?
*   **What is the overall concept or vision for TimeEcho?**  Is it a website, a mobile app, a desktop program, something else entirely?
*   **What technologies are you considering using?** (e.g., Python, JavaScript, React, databases, etc.)
*   **What roles are you looking for me to fill?**  Are you looking for help with brainstorming, writing code, designing the UI, creating content, something else?

The more information you give me, the better I can assist you.  I'm excited to hear about your project! Let's build something great.



In [4]:
def get_wikipedia_content(event_title):
    
    wiki = wikipediaapi.Wikipedia(
        user_agent='TimeEcho-HistoricalProject/1.0 (Educational Project)',  # Replace with your info
        language='en'
    )
    page = wiki.page(event_title)
    
    if page.exists():
        return {
            "title": page.title,
            "summary": page.summary,
            "full_text": page.text,
            "url": page.fullurl
        }
    else:
        return None



In [5]:
def rag_doc_preparation(data):
    data_list=[]
    for each_event in data:
        prompt=f'''Give me a JSON object in the following format for the event {each_event}, including persona-based perspectives, sensory details, historical context, and aftermath. Format it like a structured storytelling dataset.
        Output structure :
        {{
            "event_id": "unique_id",
            "event_name": "Full Name of Event",
            "time_period": "Date or Date Range",
            "location": "Geographic Location",
            "description": "Detailed 3-5 sentence description of what happened",
            "key_figures": ["Person 1", "Person 2", "Person 3"],
            "perspectives": [
                {{
                    "persona": "Type of Person Who Experienced This Event",
                    "experience": "Their unique perspective and experiences",
                    "daily_life": "How this person lived day-to-day during this time",
                    "challenges": "Specific difficulties this person would have faced"
                }},
                // Add 2-3 more perspectives
            ],
            "sensory_details": {{
                "sounds": ["Sound 1", "Sound 2", "Sound 3", "Sound 4", "Sound 5"],
                "visuals": ["Visual Element 1", "Visual Element 2", "Visual Element 3", "Visual Element 4", "Visual Element 5"],
                "environment": ["Environmental Detail 1", "Environmental Detail 2", "Environmental Detail 3", "Environmental Detail 4", "Environmental Detail 5"]
            }},
            "historical_context": "Broader historical context in 4-6 sentences",
            "aftermath": "What happened as a result in 3-4 sentences"
        }}
    
    
        Example1:
        {{
        "event_id": "berlin_wall_fall",
        "event_name": "Fall of the Berlin Wall",
        "time_period": "1989-11-09",
        "location": "Berlin, Germany",
        "description": "The fall of the Berlin Wall on November 9, 1989, marked the beginning of the end of the Cold War and the division between East and West Berlin. After weeks of civil unrest, the East German government announced that its citizens could visit West Germany and West Berlin. Crowds gathered at the wall, and with no official orders on how to handle the situation, the guards opened the gates. Throughout the night, euphoric crowds swarmed the wall, climbing on top and beginning to chip away at the concrete structure that had divided the city for 28 years.",
        "key_figures": ["Günter Schabowski", "Erich Honecker", "Mikhail Gorbachev"],
        "perspectives": [
            {{
                "persona": "East German Civilian",
                "experience": "Living under the restrictions of the German Democratic Republic (GDR) with limited freedoms, particularly regarding travel and communication with family in West Berlin.",
                "daily_life": "Waiting in long lines for basic goods, navigating government surveillance, and adapting to the economic challenges of East Germany.",
                "challenges": "Risk of imprisonment for attempting to cross the border, limited access to information, political repression."
            }},
            {{
                "persona": "Border Guard",
                "experience": "Tasked with preventing East Germans from escaping to the West, facing moral dilemmas when confronted with desperate citizens.",
                "daily_life": "Standing watch at checkpoints, following strict protocols, and potentially facing punishment for allowing unauthorized crossings.",
                "challenges": "Balancing duty with personal ethics, potential guilt from preventing family reunifications, fear of consequences for disobeying orders."
            }}
        ],
        "sensory_details": {{
            "sounds": ["Chants of 'Wir sind das Volk'", "Hammers chipping at concrete", "Border guards' radio communications", "Jubilant singing", "Honking car horns"],
            "visuals": ["Graffiti-covered concrete wall", "Checkpoint Charlie guard towers", "Families reuniting", "East German Trabant cars queuing at border crossings", "People standing atop the wall"],
            "environment": ["Cold November air", "Concrete dust", "Crowded streets", "Floodlights illuminating the wall", "Champagne bottles being opened"]
        }},
        "historical_context": "The Berlin Wall stood as a physical symbol of the Iron Curtain separating Western Europe and the Eastern Bloc during the Cold War. Built in 1961, it completely cut off West Berlin from East Berlin and East Germany. The wall was fortified with guard towers, anti-vehicle trenches, and a 'death strip' where guards were authorized to shoot escapees. By 1989, political changes in the Eastern Bloc, particularly in Hungary and Poland, created pressure for change in East Germany as well.",
        "aftermath": "The reunification of Germany followed on October 3, 1990. The wall was gradually dismantled, with sections preserved as memorials. Germany faced substantial challenges in reintegrating the two economies and societies. The fall of the wall accelerated the collapse of communist regimes across Eastern Europe and ultimately led to the dissolution of the Soviet Union in 1991."
        }}'''
        response = model.generate_content(prompt)
        data_list.append(json.loads(response.text.replace("```","").replace('json',"")))
    return data_list

In [6]:
data=[]
list_of_events=["Fall of the Berlin Wall","World War I",'World War II','Indian Independence Movement','French Revolution',"Fall of Constantinople",'Cuban Missile Crisis','Fall of Rome',"Renaissance",'March on Washington',"Boston Tea Party",'Tiananmen Square Protests',"D-Day Landings",'Assassination of JFK','First War of Independence',"Wright Brothers' First Flight"]
for each_event in list_of_events:
    event=get_wikipedia_content(each_event)
    data.append(event)
data_list=[]
data_list=rag_doc_preparation(data)

In [7]:
print(data_list[0])

{'event_id': 'berlin_wall_fall', 'event_name': 'Fall of the Berlin Wall', 'time_period': 'November 9, 1989', 'location': 'Berlin, Germany', 'description': 'The Fall of the Berlin Wall on November 9, 1989, marked a pivotal moment in world history, signifying the collapse of the Iron Curtain and the end of the Cold War division of Europe. East German authorities unexpectedly announced relaxed travel restrictions, leading thousands of citizens to flock to the wall demanding passage. Overwhelmed border guards eventually opened the gates, allowing euphoric crowds from both sides to unite.  This event triggered widespread celebrations and the eventual dismantling of the wall, symbolizing newfound freedom and the impending reunification of Germany.', 'key_figures': ['Günter Schabowski', 'Harald Jäger', 'Helmut Kohl'], 'perspectives': [{'persona': 'East German University Student', 'experience': 'Growing up in a society with limited freedoms and controlled information, constantly aware of the p

Prepare historical documents for the RAG system
This dataset is designed to serve as a retrieval base for a RAG system that can answer questions about history—not just factually, but contextually—grounded in real human experiences and primary sources.

In [8]:
def prepare_historical_documents(event_data):
    documents = []
    metadata_list = []
    ids = []
    i=0
    for event in event_data:
        i=i+1
        
        event_name=event["event_name"]  
        
        combined_text = f"""
        Event: {event.get('event_name')}
        Time Period: {event.get('time_period', 'Unknown')}
        Location: {event.get('location', 'Unknown')}
        
        Description: {event.get('description', '')}
        
        Historical Context: {event.get('historical_context', '')}
       
        Perspectives:
        """
        
        for perspective in event.get('perspectives', []):
            combined_text += f"""
            Persona: {perspective.get('persona', '')}
            Experience: {perspective.get('experience', '')}
            Daily Life: {perspective.get('daily_life', '')}
            Challenges: {perspective.get('challenges', '')}
            """
        
        combined_text += f"""
        Sensory Details:
        Sounds: {', '.join(event.get('sensory_details', {}).get('sounds', []))}
        Visuals: {', '.join(event.get('sensory_details', {}).get('visuals', []))}
        Environment: {', '.join(event.get('sensory_details', {}).get('environment', []))}
        
        Aftermath: {event.get('aftermath', '')}
        """
        
        documents.append(combined_text)
        
        metadata_list.append({
            "event_name": event.get('event_name', event_name),
            "time_period": event.get('time_period', 'Unknown'),
            "location": event.get('location', 'Unknown'),
            "key_figures": event.get('key_figures', []),
            "personas": [p.get('persona') for p in event.get('perspectives', [])]
        })
        
        
        event_id = event.get('event_id', f"event_{i}")
        ids.append(event_id)
        
        with open(f"event_data_{event_id}.json", "w") as f:
            json.dump(event, f, indent=2)
            
    return documents, metadata_list, ids

In [9]:
documents, metadata_list, ids=prepare_historical_documents(data_list)

In [10]:
for model in genai.list_models():
    if "embedContent" in model.supported_generation_methods:
        print(model.name)


models/embedding-001
models/text-embedding-004
models/gemini-embedding-exp-03-07
models/gemini-embedding-exp


In [11]:
def get_gemini_embedding(text, task_type="retrieval_document"):
    
        result = genai.embed_content(
            model="models/embedding-001",
            content=text,
            task_type=task_type
        )
        return np.array(result["embedding"], dtype=np.float32)
    

We will set up a RAG pipeline using FAISS to index historical document embeddings and a language model to generate context-aware responses.


In [12]:
def setup_faiss_rag_system(documents, metadata_list, ids):
    
    print("Generating embeddings...")
    embeddings = []
    
    for doc in documents:
        embedding = get_gemini_embedding(doc)
        embeddings.append(embedding)
    
    
    embeddings_array = np.array(embeddings, dtype=np.float32)
    
    dimension = embeddings_array.shape[1]
    
    index = faiss.IndexFlatL2(dimension)
    
    index.add(embeddings_array)
    
    faiss.write_index(index, "timeecho_faiss_index.bin")
    
    with open("timeecho_documents.pkl", "wb") as f:
        pickle.dump(documents, f)
    
    with open("timeecho_metadata.pkl", "wb") as f:
        pickle.dump(metadata_list, f)
    
    with open("timeecho_ids.pkl", "wb") as f:
        pickle.dump(ids, f)
   
    document_mapping = {
        "documents": documents,
        "metadata": metadata_list,
        "ids": ids
    }
    
    print(f"Added {len(documents)} documents to the FAISS index")
    return index, document_mapping

Load the FAISS index and document data

In [13]:
def load_faiss_system():
    
    index = faiss.read_index("timeecho_faiss_index.bin")
    
    with open("timeecho_documents.pkl", "rb") as f:
        documents = pickle.load(f)
    
    with open("timeecho_metadata.pkl", "rb") as f:
        metadata = pickle.load(f)
    
    with open("timeecho_ids.pkl", "rb") as f:
        ids = pickle.load(f)
    
    document_mapping = {
        "documents": documents,
        "metadata": metadata,
        "ids": ids
    }
    
    return index, document_mapping


Query the FAISS system to retrieve relevant historical information

In [14]:
def query_faiss_system(index, document_mapping, query, n_results=3):
    
    
    query_embedding = get_gemini_embedding(query, task_type="retrieval_query")
    
    query_embedding = np.array([query_embedding], dtype=np.float32)
    
    distances, indices = index.search(query_embedding, n_results)
    
    
    retrieved_documents = []
    retrieved_metadata = []
    retrieved_ids = []
    
    for idx in indices[0]:
        
        if idx >= 0 and idx < len(document_mapping["documents"]):
            retrieved_documents.append(document_mapping["documents"][idx])
            retrieved_metadata.append(document_mapping["metadata"][idx])
            retrieved_ids.append(document_mapping["ids"][idx])
    
    results = {
        "documents": retrieved_documents,
        "metadata": retrieved_metadata,
        "ids": retrieved_ids,
        "distances": distances[0].tolist()
    }
    
    return results


 Generate a complete TimeEcho experience

In [15]:
def generate_timeecho_experience(event, persona, rag_results=None):
   
    
    
    context = ""
    if rag_results and len(rag_results['documents']) > 0:
        context = "\n\n".join(rag_results['documents'])
    
    
    prompt = f"""
    You are TimeEcho, an immersive historical storytelling system. Create a deeply immersive first-person narrative for the following:
    
    Historical Event: {event}
    Persona: {persona}
    
    {"Historical Context from Database: " + context if context else ""}
    
    Generate a complete immersive experience in the following JSON format:
    
    {{
        "title": "A compelling title for this historical experience",
        "event_summary": "A brief factual summary of the event (2-3 sentences)",
        "persona_details": {{
            "name": "A historically plausible name for this persona",
            "background": "Brief background of this character (2-3 sentences)"
        }},
        "narrative": "A detailed first-person narrative from the perspective of the persona experiencing the historical event (700-900 words). Make it emotionally engaging, historically accurate, and immersive.",
        "sensory_details": {{
            "sights": ["5-7 specific visual elements the persona would see"],
            "sounds": ["5-7 specific sounds the persona would hear"],
            "smells": ["3-5 specific scents present in the environment"],
            "tactile": ["2-3 physical sensations the persona would feel"]
        }},
        "emotional_journey": ["5-7 emotional states the persona experiences throughout the event"],
        "historical_context": "Broader historical context for this event (150-200 words)",
        "media_elements": {{
            "image_prompts": ["3 detailed image generation prompts that would visualize key moments"],
            "audio_cues": ["3 specific sound effect or ambient audio descriptions"],
            "music_suggestions": ["2 styles of music that would enhance the emotional impact"]
        }}
    }}
    
    Ensure historical accuracy while creating an emotionally resonant experience. Be detailed and specific in your descriptions.
    """
    
    try:
        response = model.generate_content(prompt)
           
        result = response.text.replace("```json", "").replace("```", "").strip()
        
        experience_data = json.loads(result)
        return experience_data
    except Exception as e:
        print(f"Error generating TimeEcho experience: {e}")
        return None


Generate a complete TimeEcho experience
This function represents the heart of the TimeEcho experience—allowing users to engage with history not just as passive observers, but as participants in a dialogue with the past.

In [16]:
def generate_timeecho_experience(event, persona, rag_results=None):
    
    
    
    context = ""
    if rag_results and len(rag_results['documents']) > 0:
        context = "\n\n".join(rag_results['documents'])
    
    
    prompt = f"""
    You are TimeEcho, an immersive historical storytelling system. Create a deeply immersive first-person narrative for the following:
    
    Historical Event: {event}
    Persona: {persona}
    
    {"Historical Context from Database: " + context if context else ""}
    
    Generate a complete immersive experience in the following JSON format:
    
    {{
        "title": "A compelling title for this historical experience",
        "event_summary": "A brief factual summary of the event (2-3 sentences)",
        "persona_details": {{
            "name": "A historically plausible name for this persona",
            "background": "Brief background of this character (2-3 sentences)"
        }},
        "narrative": "A detailed first-person narrative from the perspective of the persona experiencing the historical event (700-900 words). Make it emotionally engaging, historically accurate, and immersive.",
        "sensory_details": {{
            "sights": ["5-7 specific visual elements the persona would see"],
            "sounds": ["5-7 specific sounds the persona would hear"],
            "smells": ["3-5 specific scents present in the environment"],
            "tactile": ["2-3 physical sensations the persona would feel"]
        }},
        "emotional_journey": ["5-7 emotional states the persona experiences throughout the event"],
        "historical_context": "Broader historical context for this event (150-200 words)",
        "media_elements": {{
            "image_prompts": ["3 detailed image generation prompts that would visualize key moments"],
            "audio_cues": ["3 specific sound effect or ambient audio descriptions"],
            "music_suggestions": ["2 styles of music that would enhance the emotional impact"]
        }}
    }}
    
    Ensure historical accuracy while creating an emotionally resonant experience. Be detailed and specific in your descriptions.
    """
    
    try:
        model = genai.GenerativeModel(model_name="gemini-2.0-flash")
        response = model.generate_content(prompt)
        #print("response: ",response)
        #print("Done2")
        
        result = response.text.replace("```json", "").replace("```", "").strip()
        
        experience_data = json.loads(result)
        return experience_data
    except Exception as e:
        print(f"Error generating TimeEcho experience: {e}")
        return None

In [17]:
def setup_timeecho_rag_faiss():
       print("Setting up FAISS system...")
    index, document_mapping = setup_faiss_rag_system(documents, metadata_list, ids)
    
    with open("timeecho_collection_info.json", "w") as f:
        json.dump({
            "event_ids": ids,
            "events": [m["event_name"] for m in metadata_list]
        }, f, indent=2)
    
    print("TimeEcho FAISS RAG system is ready!")
    return index, document_mapping

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 3)

Handle user requests for TimeEcho experiences

In [ ]:
def timeecho_experience_handler(index, document_mapping, event, persona):
    
   
    query = f"Event: {event} Persona: {persona}"
    rag_results = query_faiss_system(index, document_mapping, query)
    
   
    if rag_results["distances"] and rag_results["distances"][0] < 1.0:  
        
        experience = generate_timeecho_experience(event, persona, rag_results)
        
    else:
        
        experience = generate_timeecho_experience(event, persona)
    
    return experience


In [ ]:
def process_image_input(image_file):
    try:
        img=image_file
        model=genai.GenerativeModel('gemini-2.0-flash')
        image_parts=image_parts = [{"mime_type": "image/jpeg", "data": image_to_bytes(img)}]
        prompt = """
        Analyze this historical image and extract the following information:
        1. What historical event or time period does this image likely depict?
        2. What are the key visual elements, people, or artifacts visible?
        3. What historical context can you derive from this image?
        
        Format your response as JSON:
        {
            "event": "Name of the identified historical event or period",
            "time_period": "Approximate year or date range",
            "location": "Likely geographical location",
            "key_elements": ["List of 3-5 important visual elements"],
            "historical_context": "Brief historical context (2-3 sentences)"
        }
        """
        response = model.generate_content([prompt, image_parts[0]])
        json_match = re.search(r'```json\s*(.*?)\s*```', response.text, re.DOTALL)
        if json_match:
            result = json.loads(json_match.group(1))
        else:
            
            try:
                result = json.loads(response.text)
                print(result)
            except:
                
                result = {
                    "event": "Unknown",
                    "time_period": "Unknown",
                    "location": "Unknown",
                    "key_elements": [],
                    "historical_context": response.text[:200]
                }
        
        return result
    except Exception as e:
        print(f"Error processing image: {e}")
        return {
            "event": "Error analyzing image",
            "time_period": "Unknown",
            "location": "Unknown",
            "key_elements": [],
            "historical_context": f"Error: {str(e)}"
        }

    

In [ ]:
def image_to_bytes(image):
    """Convert PIL Image to bytes"""
    buffered = io.BytesIO()
    image.save(buffered, format="JPEG")
    return buffered.getvalue()

Generate speech from text with voice variations based on persona

In [ ]:
from IPython.display import Audio

def generate_speech(text, voice_type="neutral"):
    
    try:
        # Map voice_type to gTTS language and properties
        voice_mapping = {
            "neutral": {"lang": "en", "tld": "com"},
            "british": {"lang": "en", "tld": "co.uk"},
            "american": {"lang": "en", "tld": "us"},
            "french_accent": {"lang": "en", "tld": "ie"},  
            "german_accent": {"lang": "en", "tld": "ca"},  
            "elderly": {"lang": "en", "tld": "com.au"}, 
            "child": {"lang": "en", "tld": "co.in"},  
        }
        voice_settings = voice_mapping.get(voice_type.lower(), voice_mapping["neutral"])
        tts = gTTS(text=text, lang=voice_settings["lang"], tld=voice_settings["tld"], slow=False)
        temp_file = f"temp_speech_{voice_type}.mp3"
        tts.save(temp_file)
        
        display(Audio(temp_file, autoplay=False))
        
        return temp_file
    except Exception as e:
        print(f"Error generating speech: {e}")
        return None

Extract the historical event and persona from the query

In [ ]:
def extract_event_and_persona(query):
        
        prompt = f"""
        Analyze this query about a historical event and extract:
        1. The specific historical event being referenced
        2. The persona or viewpoint requested (if any)
        3. The type of information being requested
        
        Query: {query}
        
        Format your response as JSON:
        {{
            "event": "Name of the historical event (or 'unknown' if not specified)",
            "persona": "The requested persona/viewpoint (or 'general' if not specified)",
            "request_type": "narrative|factual|sensory|comparison"
        }}
        """
        
        try:
            model = genai.GenerativeModel(model_name="gemini-2.0-flash")
            response = model.generate_content(prompt)
            result = json.loads(response.text.replace("```json", "").replace("```", "").strip())
            return result
        except Exception as e:
            print(f"Error extracting event and persona: {e}")
            
            words = query.lower().split()
            event_keywords = ["war", "revolution", "independence", "battle", "fall", "crisis"]
            
            event = "unknown"
            for kw in event_keywords:
                if kw in words:
                    start_idx = max(0, words.index(kw) - 3)
                    end_idx = min(len(words), words.index(kw) + 4)
                    event = " ".join(words[start_idx:end_idx])
                    break
                    
            return {
                "event": event,
                "persona": "general",
                "request_type": "narrative"
            }

Generate a historical narrative

In [ ]:
!pip install gtts
from gtts import gTTS
from IPython.display import Audio
from IPython.display import Image as IPImage, display

def generate_narrative(event, persona, rag_results=None):
        
       
        context = ""
        if rag_results and len(rag_results['documents']) > 0:
            context = "\n\n".join(rag_results['documents'])
            
        
        voice_type = "neutral"
        if "soldier" in persona.lower() or "warrior" in persona.lower():
            voice_type = "american"
        elif "noble" in persona.lower() or "aristocrat" in persona.lower():
            voice_type = "british"
        elif "child" in persona.lower():
            voice_type = "child"
        elif "elder" in persona.lower() or "old" in persona.lower():
            voice_type = "elderly"
        elif any(nationality in persona.lower() for nationality in ["french", "france"]):
            voice_type = "french_accent"
        elif any(nationality in persona.lower() for nationality in ["german", "germany"]):
            voice_type = "german_accent"
        
        prompt = f"""
        You are TimeEcho, an immersive historical storytelling system. Create a deeply immersive first-person narrative for the following:
        
        Historical Event: {event}
        Persona: {persona}
        
        {"Historical Context from Database: " + context if context else ""}
        
        Generate a complete immersive experience in the following JSON format:
        
        {{
            "title": "A compelling title for this historical experience",
            "event_summary": "A brief factual summary of the event (2-3 sentences)",
            "persona_details": {{
                "name": "A historically plausible name for this persona",
                "background": "Brief background of this character (2-3 sentences)",
                "voice": "{voice_type}"
            }},
            "narrative": "A detailed first-person narrative from the perspective of the persona experiencing the historical event (500-700 words). Make it emotionally engaging, historically accurate, and immersive.",
            "sensory_details": {{
                "sights": ["5-7 specific visual elements the persona would see"],
                "sounds": ["5-7 specific sounds the persona would hear"],
                "smells": ["3-5 specific scents present in the environment"],
                "tactile": ["2-3 physical sensations the persona would feel"]
            }},
            "emotional_journey": ["5-7 emotional states the persona experiences throughout the event"],
            "historical_context": "Broader historical context for this event (150-200 words)",
            "media_elements": {{
                "image_prompts": ["3 detailed image generation prompts that would visualize key moments"],
                "audio_cues": ["3 specific sound effect or ambient audio descriptions"],
                "music_suggestions": ["2 styles of music that would enhance the emotional impact"]
            }}
        }}
        
        Ensure historical accuracy while creating an emotionally resonant experience. Be detailed and specific in your descriptions.
        """
        
        try:
            response = gemini_model.generate_content(prompt)
            result = response.text.replace("```json", "").replace("```", "").strip()
            experience_data = json.loads(result)
            
            
            if "persona_details" in experience_data and "voice" not in experience_data["persona_details"]:
                experience_data["persona_details"]["voice"] = voice_type
                
            return experience_data
        except Exception as e:
            print(f"Error generating TimeEcho experience: {e}")
            return None

Process a user query and generate a response

In [ ]:
def process_query(query, image_file=None):
        
        
        conversation_history.append({"role": "user", "content": query})
        
     
        image_analysis = None
        if image_file:
            image_analysis = process_image_input(image_file)
            print("Image Analysis:")
            print(json.dumps(image_analysis, indent=2))
        
        
        query_analysis = extract_event_and_persona(query)
        event = query_analysis["event"]
        persona = query_analysis["persona"]
        request_type = query_analysis["request_type"]
       
        if image_analysis and image_analysis["event"] != "Unknown":
            if event == "unknown":
                event = image_analysis["event"]
      
            query += f" Additional context: The image shows {image_analysis['event']} from {image_analysis['time_period']} in {image_analysis['location']}."
        

        rag_results = query_rag(query)
        
        if request_type == "factual":
            response = generate_factual_response(query, rag_results)
            response_data = {"type": "factual", "content": response}
        else:
            
            experience = generate_narrative(event, persona, rag_results)
            
            
            image_files = []
            if experience and "media_elements" in experience and "image_prompts" in experience["media_elements"]:
                for i, prompt in enumerate(experience["media_elements"]["image_prompts"]):
                    if i < 1:  
                        image_file = generate_historical_image(prompt)
                        if image_file:
                            image_files.append(image_file)
            
           
            speech_file = None
            if experience and "narrative" in experience:
                voice_type = "neutral"
                if "persona_details" in experience and "voice" in experience["persona_details"]:
                    voice_type = experience["persona_details"]["voice"]
                
                
                narrative_text = experience["narrative"]
                speech_excerpt = " ".join(narrative_text.split()[:100])  
                speech_file = generate_speech(speech_excerpt, voice_type)
            
            response_data = {
                "type": "narrative",
                "experience": experience,
                "image_files": image_files,
                "speech_file": speech_file
            }
        
       
        conversation_history.append({"role": "assistant", "content": json.dumps(response_data)})
        
        return response_data

Generate a factual response to a historical query

In [ ]:
def generate_factual_response(query, rag_results):
        
        context = ""
        if rag_results and len(rag_results['documents']) > 0:
            context = "\n\n".join(rag_results['documents'])
        
        prompt = f"""
        As TimeEcho, a historical information system, provide an accurate and informative response to this query:
        
        Query: {query}
        
        {"Based on the following historical information:" + context if context else "Using your historical knowledge:"}
        
        Provide a well-structured, factual response that is historically accurate and answers the query directly.
        Include key dates, figures, and relevant context. Format your response in clear paragraphs.
        """
        
        try:
            response = gemini_model.generate_content(prompt)
            return response.text
        except Exception as e:
            print(f"Error generating factual response: {e}")
            return "I'm sorry, I couldn't generate a response to your historical query."


Main function to run the TimeEcho system in Kaggle environment

In [ ]:
def main():
    
    print("🕰️ Initializing TimeEcho - Immersive Historical Storytelling System...")
    
    
    if os.path.exists("timeecho_faiss_index.bin"):
        print("Loading existing FAISS index...")
        index, document_mapping = load_faiss_system()
    else:
        
        print("Setting up new FAISS index...")
        index, document_mapping = setup_timeecho_rag_faiss()
    
    def process_historical_query(query, image_file=None):
        print(f"Processing query: {query}")
        
        
        if image_file:
            image_analysis = process_image_input(image_file)
            print("\nImage Analysis:")
            print(json.dumps(image_analysis, indent=2))
            
            
            if image_analysis["event"] != "Unknown":
                query += f" Additional context: The image shows {image_analysis['event']} from {image_analysis['time_period']}."
        
        result = extract_event_and_persona(query)
        event=result['event']
        persona=result['persona']
        request_type=result['request_type']
        print(f"\nIdentified Event: {event}")
        print(f"Identified Persona: {persona}")
        print(f"Request Type: {request_type}")
        
        
        rag_results = query_faiss_system(index, document_mapping, query)
        
        
        if request_type == "factual":
            
            factual_response = generate_factual_response(query, rag_results)
            print("\n📚 Historical Information:")
            print("-" * 80)
            print(factual_response)
            return {"type": "factual", "content": factual_response}
        else:
            
            experience = timeecho_experience_handler(index, document_mapping, event, persona)
            
            if experience:
                print(f"\n🕰️ TimeEcho Experience: {experience['title']}")
                print("-" * 80)
                print(f"Historical Event: {experience['event_summary']}")
                print("-" * 80)
                print(f"Through the eyes of: {experience['persona_details']['name']} - {experience['persona_details']['background']}")
                
           
                # image_file = None
                # if "media_elements" in experience and "image_prompts" in experience["media_elements"]:
                #     prompt = experience["media_elements"]["image_prompts"][0]
                #     print("\n🖼️ Generating historical visualization...")
                #     image_file = generate_historical_image(prompt)
                #     print(image_file)
                #     display(IPImage(filename=image_file))


                
                
                speech_file = None
                if "narrative" in experience:
                    voice_type = "neutral"
                    if "persona_details" in experience and "voice" in experience.get("persona_details", {}):
                        voice_type = experience["persona_details"]["voice"]
                    
                    
                    narrative_text = experience["narrative"]
                    speech_excerpt = " ".join(narrative_text.split()[:100])  
                    print("\n🔊 Generating narrative speech...")
                    speech_file = generate_speech(speech_excerpt, voice_type)
                
                print("\n📜 Narrative:")
                print(experience['narrative'])
                print("-" * 80)
                print("Sensory Details:")
                print("👁️ Sights:", ", ".join(experience['sensory_details']['sights']))
                print("👂 Sounds:", ", ".join(experience['sensory_details']['sounds']))
                print("👃 Smells:", ", ".join(experience['sensory_details']['smells']))
                print("🖐️ Tactile:", ", ".join(experience['sensory_details']['tactile']))
                print("-" * 80)
                print("Historical Context:")
                print(experience['historical_context'])
                
                return {
                    "type": "narrative",
                    "experience": experience,
                    "image_file": image_file,
                    "speech_file": speech_file
                }
            else:
                print("Failed to generate a complete historical experience.")
                return {"type": "error", "message": "Failed to generate experience"}
    
    
    print("\n=== TimeEcho System Ready ===")
    print("You can now use the process_historical_query function with any query")
    print("Example: result = process_historical_query('Tell me about World War II from a soldier's perspective')")
    
    sample_query = "Tell me about the Fall of the Berlin Wall from the perspective of an East German citizen"
    print(f"\nRunning test query: '{sample_query}'")
    result = process_historical_query(sample_query)
    
    return {
        "index": index,
        "document_mapping": document_mapping,
        "process_query": process_historical_query 
    }

if __name__ == "__main__":
    timeecho_system = main()
    

🧭 Conclusion
History is more than a timeline of events—it's a mosaic of lived experiences, emotions, and untold stories. With TimeEcho, the goal is not just to present history, but to humanize it—transforming passive information into immersive moments that bridge the gap between past and present.

By blending data, narrative, and interactive design, we can create tools that don’t just teach history, but allow us to feel it. TimeEcho is a step toward that vision—a future where technology deepens our connection to the human stories that shaped our world.

This is just the beginning. As we continue to explore the intersection of storytelling and data, one thing becomes clear: the past has a voice. And it's time we listened.




Numbers tell part of the story. But to truly feel the heartbeat of the past, we need narrative, emotion, and vision. I wrote a blog post capturing the essence behind this project—why it matters, and what it could mean.

Read it here → [TimeEcho on Medium](http://https://medium.com/@stuti2718/timeecho-experience-history-through-the-eyes-of-those-who-lived-it-744c0beda0c6)